# 🎓 Phiversity - Deploy in Google Colab
Deploy the Phiversity physics animation project directly in Google Colab with AI-powered video generation!

## 1️⃣ Mount Google Drive
Mount your Google Drive to save and access files persistently across sessions.

In [2]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
print("✅ Google Drive mounted at /content/drive")

Mounted at /content/drive
✅ Google Drive mounted at /content/drive


## 2️⃣ Clone Repository from GitHub
Clone the Phiversity repository and navigate to the project directory.

In [3]:
import subprocess
import os

# Clone the repository
repo_url = "https://github.com/sudish80/Phiversity.git"
project_dir = "/content/Phiversity"

if not os.path.exists(project_dir):
    print(f"Cloning repository from {repo_url}...")
    subprocess.run(["git", "clone", repo_url, project_dir], check=True)
    print("✅ Repository cloned successfully")
else:
    print(f"✅ Repository already exists at {project_dir}")

# Change to project directory
os.chdir(project_dir)
print(f"📂 Working directory: {os.getcwd()}")

Cloning repository from https://github.com/sudish80/Phiversity.git...
✅ Repository cloned successfully
📂 Working directory: /content/Phiversity


## 3️⃣ Install Dependencies
Install all required packages for Manim, voice synthesis, LLM integration, and FastAPI.

In [4]:
import subprocess
import sys

print("📦 Installing system dependencies...")
# Install system packages needed for Manim, ffmpeg, LaTeX, etc.
subprocess.run(["apt-get", "update"], check=True, capture_output=True)
subprocess.run([
    "apt-get", "install", "-y",
    "ffmpeg",
    "texlive-latex-base",
    "texlive-fonts-recommended",
    "texlive-latex-extra",
    "libcairo2-dev",
    "libpango1.0-dev",
    "espeak-ng"
], check=True, capture_output=True)
print("✅ System dependencies installed")

print("\n📦 Installing Python packages...")
# Install Python requirements
with open("requirements.txt", "r") as f:
    requirements = [line.strip() for line in f if line.strip() and not line.startswith("#")]

# Install in batches to avoid timeouts
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True, capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir"] + requirements, check=True)
print("✅ Python dependencies installed")

📦 Installing system dependencies...
✅ System dependencies installed

📦 Installing Python packages...
✅ Python dependencies installed


## 4️⃣ Configure Environment Variables
Set up API keys and configuration for LLM providers and other services.

⚠️ **Important**: Replace the API keys below with your own from:
- **Groq**: https://console.groq.com
- **OpenRouter**: https://openrouter.ai
- **Gemini**: https://makersuite.google.com/app/apikey
- **Elevenlabs**: https://elevenlabs.io

In [5]:
import os
from getpass import getpass

# LLM Configuration
print("🔧 Setting up environment variables...\n")

# Primary LLM Model (options: groq, openai, deepseek, gemini, ollama, openrouter)
os.environ["LLM_MODEL"] = "groq"
os.environ["USE_FINETUNED_MODEL"] = "true"

# Groq API Key
groq_key = getpass("Enter your GROQ_API_KEY (leave empty to use existing): ")
if groq_key:
    os.environ["GROQ_API_KEY"] = groq_key

# Gemini API Key (optional)
gemini_key = getpass("Enter your GEMINI_API_KEY (leave empty to skip): ")
if gemini_key:
    os.environ["GEMINI_API_KEY"] = gemini_key
    os.environ["GOOGLE_API_KEY"] = gemini_key

# OpenRouter API Key (optional)
openrouter_key = getpass("Enter your OPENROUTER_API_KEY (leave empty to skip): ")
if openrouter_key:
    os.environ["OPENROUTER_API_KEY"] = openrouter_key
    os.environ["OPENROUTER_MODEL"] = "anthropic/claude-3-sonnet"

# Voice Configuration
os.environ["VOICE_ENGINE"] = "pyttsx3"  # Options: pyttsx3, gtts, elevenlabs
os.environ["ELEVENLABS_API_KEY"] = ""  # Add if using ElevenLabs

# Manim Configuration
os.environ["VIDEO_QUALITY"] = "low_quality"  # low_quality, medium_quality, high_quality
os.environ["MANIM_QUALITY"] = "low_quality"

# Application Configuration
os.environ["OUTPUT_DIR"] = "media"
os.environ["GENERATE_AUDIO"] = "true"
os.environ["GENERATE_VIDEO"] = "true"
os.environ["JOB_TIMEOUT"] = "600"

print("✅ Environment variables configured")

🔧 Setting up environment variables...

✅ Environment variables configured


## 5️⃣ Test LLM Connection
Verify that the LLM API connection is working correctly.

In [6]:
import os
from pathlib import Path

# Test Groq API
print("🧪 Testing LLM connection with Groq...\n")

try:
    import groq
    
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        print("⚠️  GROQ_API_KEY not set. Please set it in the environment variables cell.")
    else:
        client = groq.Groq(api_key=groq_api_key)
        
        # Test simple message
        message = client.chat.completions.create(
            messages=[{"role": "user", "content": "Explain conservation of energy in one sentence."}],
            model="mixtral-8x7b-32768"
        )
        
        print("✅ Groq API Connection Successful!")
        print(f"\nSample response:\n{message.choices[0].message.content}\n")
        
except Exception as e:
    print(f"❌ Error testing Groq API: {e}")
    print("Please check your API key and try again.")

🧪 Testing LLM connection with Groq...

❌ Error testing Groq API: No module named 'groq'
Please check your API key and try again.


## 6️⃣ Run the Application
Execute the Phiversity application with the FastAPI server and generate physics animations.

The server will run on `http://localhost:8000`

In [7]:
import subprocess
import time
import os
from pathlib import Path

# Change to project directory
os.chdir("/content/Phiversity")

# Install ngrok for tunneling (make the server accessible from outside)
print("📦 Installing ngrok for public URL access...")
subprocess.run(["pip", "install", "-q", "pyngrok"], check=True)

# Generate sample problem to test
sample_problem = "Explain how the conservation of momentum applies to a collision between two billiard balls. Show the before and after momentum vectors."

print("\n🚀 Starting Phiversity FastAPI Server...")
print(f"📝 Sample problem: {sample_problem}\n")

# Create a test script
test_script = '''
import os
import sys
import asyncio
from pathlib import Path

# Set environment
os.environ["LLM_MODEL"] = os.getenv("LLM_MODEL", "groq")
os.environ["MANIM_QUALITY"] = "low_quality"

# Add project to path
sys.path.insert(0, "/content/Phiversity")

# Import the orchestrator
from scripts.orchestrator.prompt_orchestrator import call_groq_solver

print("🧠 Processing physics problem with LLM...")
print(f"Problem: {problem}")

try:
    result = call_groq_solver(problem)
    print("\\n✅ LLM Processing Complete!")
    print(f"\\nGenerated Solution:\\n{result}")
    
    # Save result
    output_dir = Path("media/texts")
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / "solution.txt"
    with open(output_file, "w") as f:
        f.write(result)
    print(f"\\n💾 Solution saved to {output_file}")
    
except Exception as e:
    print(f"\\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
'''

# Write and run test script
with open("/tmp/test_phiversity.py", "w") as f:
    f.write(test_script.replace("{problem}", f'"{sample_problem}"'))

print("Running test...")
result = subprocess.run(
    ["python", "/tmp/test_phiversity.py"],
    cwd="/content/Phiversity",
    capture_output=True,
    text=True,
    timeout=300
)

print(result.stdout)
if result.stderr:
    print("Errors/Warnings:")
    print(result.stderr)

📦 Installing ngrok for public URL access...

🚀 Starting Phiversity FastAPI Server...
📝 Sample problem: Explain how the conservation of momentum applies to a collision between two billiard balls. Show the before and after momentum vectors.

Running test...

Errors/Warnings:
  File "/tmp/test_phiversity.py", line 18
    print(f"Problem: "Explain how the conservation of momentum applies to a collision between two billiard balls. Show the before and after momentum vectors."")
          ^^^^^^^^^^^^^^^^^^^
SyntaxError: invalid syntax. Perhaps you forgot a comma?



## 6️⃣A View Generated Outputs in Colab
Display videos and results directly in the notebook!

## 7️⃣ Download Output Files
Download generated videos, audio files, and text solutions to your local machine or Google Drive.

In [8]:
import os
import shutil
from pathlib import Path
from google.colab import files

print("📁 Checking generated output files...\n")

media_dir = Path("/content/Phiversity/media")
output_files = {
    "Videos": list(media_dir.glob("videos/**/*.mp4")),
    "Audio": list(media_dir.glob("videos/**/*.wav")) + list(media_dir.glob("videos/**/*.mp3")),
    "Text Solutions": list(media_dir.glob("texts/**/*.json")) + list(media_dir.glob("texts/**/*.txt"))
}

# Display available files
for category, files_list in output_files.items():
    if files_list:
        print(f"✅ {category}:")
        for f in files_list[:5]:  # Show first 5 files
            file_size = f.stat().st_size / (1024*1024)  # Convert to MB
            print(f"   - {f.name} ({file_size:.2f} MB)")
        if len(files_list) > 5:
            print(f"   ... and {len(files_list) - 5} more files")
    else:
        print(f"⚠️  No {category.lower()} generated yet")

# Option to download files
print("\n💾 Downloading files to local machine...")
output_zip = "/tmp/phiversity_outputs.zip"

# Create zip file with all outputs
if list(media_dir.glob("**/*")):
    print("Creating archive...")
    shutil.make_archive("/tmp/phiversity_outputs", "zip", media_dir)
    print(f"✅ Archive created: {output_zip}")
    
    # Download
    print("Downloading to your computer...")
    files.download(output_zip)
    print("✅ Download complete!")
else:
    print("⚠️  No output files to download yet. Run the application first.")

📁 Checking generated output files...

⚠️  No videos generated yet
⚠️  No audio generated yet
⚠️  No text solutions generated yet

💾 Downloading files to local machine...
⚠️  No output files to download yet. Run the application first.


## 📹 View Generated Videos in Colab
Display video outputs directly in the Colab notebook with a beautiful player.

In [ ]:
from pathlib import Path
from IPython.display import Video, HTML, display, FileLink
import os

print("🎬 Looking for generated videos...\n")

media_dir = Path("/content/Phiversity/media")
videos_dir = media_dir / "videos"

# Find all MP4 files recursively
video_files = list(videos_dir.rglob("*.mp4")) if videos_dir.exists() else []

if video_files:
    print(f"✅ Found {len(video_files)} video(s)!\n")
    
    for idx, video_path in enumerate(video_files[:5], 1):  # Show first 5 videos
        print(f"🎥 Video {idx}: {video_path.name}")
        print(f"   Size: {video_path.stat().st_size / (1024*1024):.2f} MB")
        print(f"   Path: {video_path}\n")
        
        # Display video with player
        display(HTML(f"""
        <div style="margin: 20px 0; border: 2px solid #6366f1; border-radius: 12px; padding: 15px; background: rgba(99, 102, 241, 0.1);">
            <h3 style="color: #6366f1; margin-top: 0;">📹 {video_path.name}</h3>
            <video width="100%" height="auto" controls style="border-radius: 8px;">
                <source src="{str(video_path)}" type="video/mp4">
                Your browser does not support the video tag.
            </video>
            <p style="margin-bottom: 0; color: #cbd5e1; font-size: 12px;">
                Size: {video_path.stat().st_size / (1024*1024):.2f} MB
            </p>
        </div>
        """))
else:
    print("⚠️  No videos found yet. Run the 'Run the Application' cell first to generate videos.")

## 📝 View Text Solutions & Results
Display generated solution plans, analysis, and metadata.

In [ ]:
import json
from pathlib import Path
from IPython.display import display, HTML

print("📄 Looking for solution files...\n")

media_dir = Path("/content/Phiversity/media")
texts_dir = media_dir / "texts"

# Find all JSON and text files
if texts_dir.exists():
    json_files = list(texts_dir.glob("*.json"))
    txt_files = list(texts_dir.glob("*.txt"))
    
    print(f"📊 Found {len(json_files)} JSON files and {len(txt_files)} text files\n")
    
    # Display JSON solutions
    for json_file in json_files[:3]:  # Show first 3
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            print(f"✅ {json_file.name}")
            display(HTML(f"""
            <div style="background: rgba(16, 185, 129, 0.1); border: 1px solid #10b981; 
                        border-radius: 8px; padding: 15px; margin: 10px 0; font-family: monospace;">
                <h4 style="color: #10b981; margin-top: 0;">📋 Solution Data</h4>
                <pre style="white-space: pre-wrap; color: #cbd5e1; font-size: 12px; max-height: 300px; overflow-y: auto;">
{json.dumps(data, indent=2)[:1000]}...</pre>
            </div>
            """))
        except Exception as e:
            print(f"❌ Error reading {json_file.name}: {e}")
    
    # Display text solutions
    for txt_file in txt_files[:3]:
        try:
            with open(txt_file, 'r') as f:
                content = f.read()
            
            print(f"✅ {txt_file.name}")
            display(HTML(f"""
            <div style="background: rgba(99, 102, 241, 0.1); border: 1px solid #6366f1; 
                        border-radius: 8px; padding: 15px; margin: 10px 0;">
                <h4 style="color: #6366f1; margin-top: 0;">📝 {txt_file.name}</h4>
                <div style="color: #cbd5e1; font-size: 13px; max-height: 300px; overflow-y: auto; white-space: pre-wrap;">
{content[:500]}...</div>
            </div>
            """))
        except Exception as e:
            print(f"❌ Error reading {txt_file.name}: {e}")
else:
    print("⚠️  No text outputs found yet. Run the application to generate solutions.")

## 🖼️ View Generated Images
Display any generated scene images and renders.

In [ ]:
from pathlib import Path
from IPython.display import display, HTML, Image
import glob

print("🖼️  Looking for generated images...\n")

media_dir = Path("/content/Phiversity/media")
images_dir = media_dir / "images"

# Find all image files
image_files = []
if images_dir.exists():
    image_files = list(images_dir.glob("**/*.png")) + list(images_dir.glob("**/*.jpg")) + list(images_dir.glob("**/*.jpeg"))

if image_files:
    print(f"✅ Found {len(image_files)} image(s)!\n")
    
    for idx, img_path in enumerate(image_files[:10], 1):  # Show first 10
        print(f"🖼️  Image {idx}: {img_path.name}")
        
        try:
            # Display image
            display(HTML(f"""
            <div style="margin: 15px 0; border: 2px solid #ec4899; border-radius: 12px; padding: 15px; background: rgba(236, 72, 153, 0.05);">
                <h4 style="color: #ec4899; margin-top: 0;">🖼️  {img_path.name}</h4>
                <img src="{str(img_path)}" style="max-width: 100%; max-height: 500px; border-radius: 8px;">
                <p style="margin-bottom: 0; color: #cbd5e1; font-size: 12px;">
                    Size: {img_path.stat().st_size / 1024:.2f} KB
                </p>
            </div>
            """))
        except Exception as e:
            print(f"Error displaying {img_path.name}: {e}")
else:
    print("⚠️  No images found yet. Images will appear after generating animations.")

## 📊 Output Summary Dashboard
View a complete summary of all generated outputs in one place.

In [ ]:
from pathlib import Path
from IPython.display import display, HTML
import os

print("📊 Generating output summary...\n")

media_dir = Path("/content/Phiversity/media")

# Count all outputs
videos = list(media_dir.rglob("*.mp4"))
audios = list(media_dir.rglob("*.wav")) + list(media_dir.rglob("*.mp3"))
images = list(media_dir.rglob("*.png")) + list(media_dir.rglob("*.jpg")) + list(media_dir.rglob("*.jpeg"))
texts = list(media_dir.rglob("*.json")) + list(media_dir.rglob("*.txt"))

# Calculate total sizes
total_video_size = sum(f.stat().st_size for f in videos) / (1024*1024)
total_audio_size = sum(f.stat().st_size for f in audios) / (1024*1024)
total_image_size = sum(f.stat().st_size for f in images) / (1024*1024)
total_size = total_video_size + total_audio_size + total_image_size

# Create dashboard
dashboard = f"""
<div style="background: linear-gradient(135deg, rgba(99, 102, 241, 0.1), rgba(139, 92, 246, 0.1)); 
            border: 2px solid #6366f1; border-radius: 16px; padding: 30px; margin: 20px 0;">
    <h2 style="color: #60a5fa; margin-top: 0; text-align: center;">📊 Phiversity Output Dashboard</h2>
    
    <div style="display: grid; grid-template-columns: 1fr 1fr 1fr 1fr; gap: 15px; margin-top: 30px;">
        <!-- Videos -->
        <div style="background: rgba(59, 130, 246, 0.2); border: 1px solid #3b82f6; border-radius: 12px; padding: 20px; text-align: center;">
            <div style="font-size: 32px; margin-bottom: 8px;">🎬</div>
            <div style="font-size: 24px; font-weight: bold; color: #3b82f6;">{len(videos)}</div>
            <div style="color: #cbd5e1; font-size: 12px;">Videos</div>
            <div style="color: #94a3b8; font-size: 12px; margin-top: 8px;">{total_video_size:.2f} MB</div>
        </div>
        
        <!-- Audio -->
        <div style="background: rgba(168, 85, 247, 0.2); border: 1px solid #a855f7; border-radius: 12px; padding: 20px; text-align: center;">
            <div style="font-size: 32px; margin-bottom: 8px;">🔊</div>
            <div style="font-size: 24px; font-weight: bold; color: #a855f7;">{len(audios)}</div>
            <div style="color: #cbd5e1; font-size: 12px;">Audio Files</div>
            <div style="color: #94a3b8; font-size: 12px; margin-top: 8px;">{total_audio_size:.2f} MB</div>
        </div>
        
        <!-- Images -->
        <div style="background: rgba(236, 72, 153, 0.2); border: 1px solid #ec4899; border-radius: 12px; padding: 20px; text-align: center;">
            <div style="font-size: 32px; margin-bottom: 8px;">🖼️</div>
            <div style="font-size: 24px; font-weight: bold; color: #ec4899;">{len(images)}</div>
            <div style="color: #cbd5e1; font-size: 12px;">Images</div>
            <div style="color: #94a3b8; font-size: 12px; margin-top: 8px;">{total_image_size:.2f} MB</div>
        </div>
        
        <!-- Text Solutions -->
        <div style="background: rgba(16, 185, 129, 0.2); border: 1px solid #10b981; border-radius: 12px; padding: 20px; text-align: center;">
            <div style="font-size: 32px; margin-bottom: 8px;">📝</div>
            <div style="font-size: 24px; font-weight: bold; color: #10b981;">{len(texts)}</div>
            <div style="color: #cbd5e1; font-size: 12px;">Text Files</div>
            <div style="color: #94a3b8; font-size: 12px; margin-top: 8px;">Solutions</div>
        </div>
    </div>
    
    <!-- Total Size -->
    <div style="margin-top: 20px; padding: 15px; background: rgba(255, 255, 255, 0.05); border-radius: 8px; text-align: center;">
        <div style="color: #cbd5e1; margin-bottom: 8px;">Total Output Size</div>
        <div style="font-size: 24px; font-weight: bold; color: #60a5fa;">{total_size:.2f} MB</div>
    </div>
    
    <!-- Quick Links -->
    <div style="margin-top: 25px; padding: 20px; background: rgba(0, 0, 0, 0.2); border-radius: 8px;">
        <div style="color: #cbd5e1; font-weight: bold; margin-bottom: 12px;">📌 Quick Access:</div>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
            <div style="background: rgba(99, 102, 241, 0.1); padding: 10px; border-radius: 6px; text-align: center;">
                <a href="/content/Phiversity/media/videos" style="color: #60a5fa; text-decoration: none;">📁 Videos Folder</a>
            </div>
            <div style="background: rgba(99, 102, 241, 0.1); padding: 10px; border-radius: 6px; text-align: center;">
                <a href="/content/Phiversity/media/texts" style="color: #60a5fa; text-decoration: none;">📁 Solutions Folder</a>
            </div>
            <div style="background: rgba(99, 102, 241, 0.1); padding: 10px; border-radius: 6px; text-align: center;">
                <a href="/content/Phiversity/media/images" style="color: #60a5fa; text-decoration: none;">📁 Images Folder</a>
            </div>
            <div style="background: rgba(99, 102, 241, 0.1); padding: 10px; border-radius: 6px; text-align: center;">
                <a href="/content/drive/My Drive" style="color: #60a5fa; text-decoration: none;">☁️ Google Drive</a>
            </div>
        </div>
    </div>
</div>
"""

display(HTML(dashboard))

# Print summary to console
print("✅ Summary:")
print(f"   🎬 Videos: {len(videos)} ({total_video_size:.2f} MB)")
print(f"   🔊 Audio: {len(audios)} ({total_audio_size:.2f} MB)")
print(f"   🖼️  Images: {len(images)} ({total_image_size:.2f} MB)")
print(f"   📝 Text Solutions: {len(texts)}")
print(f"   📊 Total Size: {total_size:.2f} MB")

## 📥 Download Specific Files
Browse and download individual output files.

In [ ]:
from pathlib import Path
from google.colab import files
from IPython.display import display, HTML

print("📥 Preparing downloads...\n")

media_dir = Path("/content/Phiversity/media")

# Find all downloadable files
all_files = {
    "Videos": list(media_dir.rglob("*.mp4")),
    "Audio": list(media_dir.rglob("*.wav")) + list(media_dir.rglob("*.mp3")),
    "Images": list(media_dir.rglob("*.png")) + list(media_dir.rglob("*.jpg")) + list(media_dir.rglob("*.jpeg")),
    "Solutions": list(media_dir.rglob("*.json")) + list(media_dir.rglob("*.txt"))
}

# Create download buttons
html_content = '<div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 12px;">\n'

for category, file_list in all_files.items():
    if file_list:
        for file_path in file_list[:10]:  # Show first 10 of each type
            file_size_mb = file_path.stat().st_size / (1024*1024)
            safe_name = file_path.name.replace("'", "")
            
            html_content += f'''
    <div style="background: rgba(99, 102, 241, 0.1); border: 1px solid #6366f1; 
                border-radius: 8px; padding: 12px; text-align: center;">
        <div style="color: #cbd5e1; font-size: 12px; margin-bottom: 8px;">{category}</div>
        <button onclick="downloadFile('{str(file_path)}')" 
                style="background: #6366f1; color: white; border: none; padding: 8px 16px; 
                       border-radius: 6px; cursor: pointer; font-size: 12px; font-weight: bold;">
            📥 {file_path.name[:20]}...
        </button>
        <div style="color: #94a3b8; font-size: 10px; margin-top: 6px;">{file_size_mb:.2f} MB</div>
    </div>
'''

html_content += '</div>'

# Display buttons
if any(all_files.values()):
    print("✅ Available files for download:\n")
    display(HTML(html_content))
    
    # Add instructions
    display(HTML("""
    <div style="background: rgba(16, 185, 129, 0.1); border: 1px solid #10b981; 
                border-radius: 8px; padding: 15px; margin-top: 20px;">
        <h4 style="color: #10b981; margin-top: 0;">💡 How to Download</h4>
        <ol style="color: #cbd5e1; margin: 0; padding-left: 20px;">
            <li>Click any download button above</li>
            <li>The file will appear in your browser's Downloads folder</li>
            <li>Or use Ctrl+Click to copy the file path and use <code>files.download()</code></li>
        </ol>
    </div>
    """))
else:
    print("⚠️  No files available yet. Generate outputs first!")

## 🚀 BONUS: Run FastAPI Server (For Continuous Use)
Start the FastAPI server with public URL access via ngrok (requires ngrok authtoken).

In [9]:
import subprocess
import os
from getpass import getpass
from pyngrok import ngrok

# Optional: Get ngrok authtoken from https://dashboard.ngrok.com/auth
print("🔐 Setting up public URL access with ngrok...\n")
print("To enable public access:")
print("1. Visit https://dashboard.ngrok.com/auth")
print("2. Copy your authtoken")
print("3. Paste it below (leave empty to skip)\n")

ngrok_token = getpass("Enter your ngrok authtoken (or press Enter to skip): ")

if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
    print("✅ ngrok configured")
else:
    print("⚠️  Skipping ngrok - server will only be accessible locally")

print("\n🚀 Starting FastAPI Server...\n")

# Start the server in background
server_process = subprocess.Popen(
    ["python", "-m", "uvicorn", "scripts.server.app:app", "--host", "0.0.0.0", "--port", "8000"],
    cwd="/content/Phiversity",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

import time
time.sleep(5)  # Give server time to start

if ngrok_token:
    try:
        # Create ngrok tunnel
        public_url = ngrok.connect(8000)
        print(f"✅ Server running at {public_url}")
        print(f"\n📡 Public API URL: {public_url}/docs")
        print("🔗 You can now make requests to the API from anywhere!")
    except Exception as e:
        print(f"⚠️  ngrok error: {e}")
        print("Server is still running locally at http://localhost:8000")
else:
    print("✅ Server running locally at http://localhost:8000/docs")

print("\n⏱️  Server will keep running. You can now make API requests!")
print("To stop the server, run: server_process.terminate()")

🔐 Setting up public URL access with ngrok...

To enable public access:
1. Visit https://dashboard.ngrok.com/auth
2. Copy your authtoken
3. Paste it below (leave empty to skip)

⚠️  Skipping ngrok - server will only be accessible locally

🚀 Starting FastAPI Server...

✅ Server running locally at http://localhost:8000/docs

⏱️  Server will keep running. You can now make API requests!
To stop the server, run: server_process.terminate()


## 📚 Troubleshooting & Next Steps

### Common Issues

**❌ "GROQ_API_KEY not found"**
- Solution: Get your API key from https://console.groq.com
- Re-run the environment variables cell and paste your key

**❌ "ModuleNotFoundError"**
- Solution: Make sure all dependencies are installed (run the installation cell again)
- Restart the kernel: Kernel → Restart Runtime

**❌ "Error generating video"**
- Check that you have enough Colab resources (runtime may timeout on complex videos)
- Use `VIDEO_QUALITY=low_quality` for faster generation

**❌ "ngrok not working"**
- Get a free account at https://ngrok.com
- Copy your authtoken from https://dashboard.ngrok.com/auth

### Next Steps

1. ✅ All files are already committed to GitHub
2. 📝 Modify `os.environ["LLM_MODEL"]` to try different LLMs
3. 🎬 Experiment with different physics problems
4. 📊 Save outputs to Google Drive for persistent storage
5. 🌐 Share the ngrok URL with collaborators for live demos

### Useful API Endpoints

- `GET /docs` - Interactive API documentation (Swagger UI)
- `POST /run` - Generate animation for a physics problem
- `POST /test-llm` - Test LLM connection with a question

### Resources

- 📖 [Phiversity GitHub](https://github.com/sudish80/Phiversity)
- 🎓 [Manim Documentation](https://docs.manim.community/)
- 🤖 [Groq API Docs](https://console.groq.com/docs)
- 🌐 [FastAPI Docs](https://fastapi.tiangolo.com/)